In [94]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as f

In [95]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import Window

In [96]:
spark = SparkSession.builder.appName("zim").master("local[*]").config("spark.sql.broadcastTimeout", "36000").config("spark.driver.memory", "4G").getOrCreate()

In [97]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [121]:
# Dataframe de transacciones con todas las unidades 
df = spark.read.parquet("/data/zim_data/transacciones.parquet")

In [122]:
# Filtrar las bolsas plasticas de retail
df = df.filter(~f.col("PROD").startswith("BOLSA TIPO"))

In [123]:
# Agrupar los productos comprados por cada usuario y organizarlos por fecha
w = Window.partitionBy('USER').orderBy('DATE')
df = df.withColumn('PRODS', f.collect_list('PROD').over(w))
df = df.groupBy('USER').agg(f.max('PRODS').alias('PRODS'))

In [ ]:
df.head()

In [66]:
# Entrenar el modelo
model = Word2Vec(vectorSize=5, inputCol="PRODS", outputCol="model").fit(df)

In [1]:
model.findSynonymsArray("PAÑUELOS", 10)

NameError: name 'model' is not defined